# A simple model of a B-Scan 

Here we see how we can create a simple gprMax model of a B-Scan or otherwise known as a radargram or simply a GPR scan. The process of designing the model is almost the same with the exception of specifying the *starting* locations of the transmitter and receiver and then the steps that gprMax will use in subsequent runs to move them in order to generate the simulated scan data one trace at the time. Obviously, the targets do not move!

__EXECUTE THE FOLLOWING CELL BEFORE ANY ELSE!!!__

In [ ]:
run ../setup/preamble.ipynb

Consider another simple 2D model. We again assume that $z$-direction is infinite long and in the model it only has one cell width. So, let us consider an $x-y$ model of 0.6 metres by 0.3 metres and a spatial step of $\Delta x = \Delta y =$ 0.002 metres.

We initially setup the source and receiver close to the edge of the model __but not very close to the boundaries of the grid and at least 10 cells away__ and over a half space of a simple material like sand and we will set the relative permitivity of it to $\epsilon_r$ = 3.5. We also will assume that there are no boundaries to this model for our waves and so absorbing boundaries are used by default around the edges of our numerical grid.

We then will specify a cylindrical target at a depth of about 0.1 metres from the interface having a radius of 0.01 metres.

Our scan will have 51 traces each separated by 0.008 metres. The first transmitter and receiver pair will be placed at (0.096, 0.256, 0) and (0.14, 0.256, 0) respectively.

We will run the model for a simulated time of 7 nanoseconds and use a Hertzian dipole source with a centre frequency of 2000 MHz and amplitutde 1. The antennna current waveform has the shape of a gaussian derivative.

We will call this model __simplebscan.in__ and we can save this file using the cell below

In [ ]:
%%writefile simplebscan.in
#title: A simple B-Scan
#domain: 0.6 0.3 0.002
#dx_dy_dz: 0.002 0.002 0.002
#time_window: 7e-9
#messages: y
#num_threads: 1 

#material: 3.5 0 1 0 sand

#waveform: gaussiandot 1 2000e6 mypulse
#hertzian_dipole: z 0.096 0.256 0 mypulse
#rx: 0.14 0.256 0

#src_steps: 0.008 0 0
#rx_steps: 0.008 0 0
#box: 0 0 0 0.6 0.25 0.002 sand
#cylinder: 0.3 0.15 0 0.3 0.15 0.002 0.01 pec
#box: 0 0 0 0.6 0.03 0.002 free_space

#geometry_view: 0 0 0 0.6 0.3 0.002 0.002 0.002 0.002 simplebscan n

We can see the input file we generated running the cell below!

In [ ]:
# View input file for gprMax

view_file('simplebscan.in')

We run gprMax with the __-n NumberOfTraces__ flag to let it know how many runs we want!

In [ ]:
!python -m gprMax simplebscan.in -n 4

In [ ]:
# View the 2D gprMax model to see if it is OK Remember that every model has a number at the end for each trace.

gprMax_model('simplebscan1.vti')

We can collect all the traces using a new function to read a B-scan but don't give the file extention!

In [ ]:
# the filename will be the input file base neame with a ".out" extension 
Bscan, time = gprMax_Bscan('simplebscan.out', 1, 'Ez')

We can plot the results using the Bscan plotting function

In [ ]:
# Plot B-scan - plot_Bscan(bscan, time_vector, colormap, time_offset) We do not have to use the time_offset if it is 0

plot_Bscan(Bscan, time, cmap='seismic', time_offset=0)

Finally, if we want to export this Bscan using the same file format (.dzt) as used by the GSSI radar system so we can be able to import it into the processing software we will use for dealing with real GPR data, we can use the following __gprMax_to_dzt__ function

In [ ]:
# gprMax_to_dzt('fileName', rx_number, 'component', FrequencyMHz2000, Tx_Rx_separtion, AScan_step_size)
gprMax_to_dzt('simplebscan', 1, 'Ez', 2000, 0.08, 0.008)